# Golemity

Golemity consists of a tribe of golems with health, status, and age, and the ability to speak, listen, and choose each others' conversation partners, the first conversation partners of young golems being their parent. 

Their initial values are random, but they make an interesting system that has the possibility to thrive or collapse under certain circumstances.

In [ ]:
var Constants = {
  speechBonus: 0.1,
  listenBonus: 0.1,
}

function choice(l=[]){
  return l[Math.floor(Math.random() * l.length)];
}

function mean(l=[]){
  var sum = 0.0;
  for(var i=0; i<l.length; i++){
  	sum += l[i];
  }
  return sum/l.length;
}

function Golem(){
  this.health = Math.random();
  this.lapses = 0.0;
}

Golem.prototype.speak = function(){
  this.health += Math.random() * Constants.speechBonus;
  return Math.random();
}

Golem.prototype.age = function(){
  this.lapses += 1.0;
}

Golem.prototype.listenTo = function(speech){
  var impact = Math.random() * speech * Constants.listenBonus;
  this.health *- impact
}

In [ ]:
function Setting(n){
  this.golems = [];
  for(var i=0; i<n; i++){
    this.golems.push(new Golem());
  }
  
  console.log(`Setting created, <br>Number of Golems: ${this.golems.length} <br>Mean Health: ${mean(this.golems.map(g => g.health))}.`)
}


In [ ]:
var ROUNDS = 100;
var GOLEMS = 10;

Setting.prototype.historize = function(fact={}){
  if(this.history === undefined){
  	this.history = [];
  }
  
  for(var attribute of Object.keys(fact)){
  	if(this.history[attribute] === undefined){
      this.history[attribute] = [];
    }
    
  	this.history[attribute].push(fact[attribute]);
  }
}

var setting = new Setting(GOLEMS);

for(var i=0; i<ROUNDS; i++){
  var speaker = choice(setting.golems);
  var listener = choice(setting.golems);
  
  var speech = speaker.speak();
  listener.listenTo(speech);
  
  setting.historize({round: i, 'mean health': mean(setting.golems.map(g => g.health))})
}

Setting.prototype.round = function(){
	var speaker = choice(this.golems);
  var listener = choice(this.golems);
  
  var speech = speaker.speak();
  listener.listenTo(speech);
  
  var meanHealth = mean(this.golems.map(g => g.health))
  
  this.historize({
  	'round': this.history.round.length - 1, 
    'mean health': meanHealth
  });
	return mean(this.golems.map(g => g.health));
}

setting.history['mean health']

var vlSpec = {
  $schema: 'https://vega.github.io/schema/vega-lite/v5.json',
  description: 'A simple bar chart with embedded data.',
  data: {
  	values: setting.history.round.map(h => {
    	return {
      	round: h, 
        'mean health': setting.history['mean health'][h]
      };
		})
  },
  mark: 'line',
  encoding: {
    x: {field: 'round', type: 'ordinal'},
    y: {field: 'mean health', type: 'quantitative'}
  }
};

vlSpec

In [ ]:
var html = `<div id="chart" style="display: block;"></div>`
html

In [ ]:
var chart = document.querySelector('es-notebook').shadowRoot.querySelectorAll('notebook-cell')[4].shadowRoot.querySelector('#chart');

vegaEmbed(chart, {
  $schema: 'https://vega.github.io/schema/vega-lite/v5.json',
  data: {name: 'table'},
  width: 400,
  mark: 'point',
  encoding: {
    x: {field: 'round', type: 'ordinal', scale: {zero: false}},
    y: {field: 'mean health', type: 'quantitative'}
  }
}).then(function(res) {
  /**
   * Generates a new tuple with random numbers in the range 0, 10.
   */
  function newGenerator(){
	  var counter = -1;
    var previousMeanHealth = [0, 0, 0, 0, 0];
  
    return function(){
    	counter++;
    	var newVals = previousMeanHealth.map(function(v, c){     
        return {
        	round: counter,
          'mean health': setting.round()
        }
      });
      
      previousMeanHealth = newVals.map(function(v){
        return v['mean health'];
      });
      
      return newVals;
    };
  }

  var valueGenerator = newGenerator();
  var minimumRound = -20;
  window.setInterval(function () {
    minimumRound++;
    var changeSet = vega
      .changeset()
      .insert(valueGenerator())
      .remove(function (t) {
        return t['round'] < minimumRound;
      });
    res.view.change('table', changeSet).run();
  }, 100);
});

setting.historize({round: 0})